# Milestones 3
## Covariance & Correlation
### Prepared by:
HAFIFI BIN YAHYA - WQD170042                                          
NOR ASMIDAH BINTI MOHD ARSHAD - WQD180006                             
MAS RAHAYU BINTI MOHAMAD - WQD180048                                  
LEE CHUN MUN - WQD180066                                              
JOJIE ANAK NANJU - WQD180029 

## Loading Data
The stock data is obtained from web scrapping as well as from stock's historical website (Investing.com) for stock history before Feb 2019 (Remark: We started scrapping from Feb).

In [1]:
# Importing related packages or libraries.

import pandas as pd
import numpy as np
import os


In [2]:
# Load stock data (EOD) for all "Telecommunications & Media" companies that listed in KLSE Main Board.
# The data is obtained from the scraping activities during Milestone 1 and 2. 
# In this Milestone 3, we will loaded the data to stock dataframe for analysis.

list_of_companies = ['Amtel_Telco', 'AsiaMedia_Media', 'Astro_Media', 'Axiata_Telco', 'BerjayaMedia_Media', 'Digi_Telco',
                     'EcoBuilt_Media', 'GreenPacket_Telco', 'Maxis_Telco', 'MediaChineseInternational_Media', 'MediaPrima_Media',
                     'OCKGroup_Telco', 'Pelangi_Media', 'Sasbadi_Media', 'SeniJaya_Media', 'Star_Media', 'TimeDotCom_Telco',
                     'TM_Telco', 'Utusan_Media']

stock = pd.DataFrame({"Date":[],"Price":[],"Open":[],"High":[],"Low":[],"Volume":[],"Change%":[],"Company":[]})
stock = stock.set_index('Date')

# Loaded all stock information for respective companies to stock dataframe.
for company in list_of_companies:
    path = "Data/" + company + ".csv"
    df = pd.read_csv(path, sep=",", header=0, parse_dates=[0])
    df.columns =["Date","Price","Open","High","Low","Volume","Change%"]
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    df["Company"] = company
    stock = stock.append(df)

stock.head(10)


,Price,Open,High,Low,Volume,Change%,Company
Date,,,,,,,
2019-03-22,0.665,0.665,0.665,0.665,1.00K,0.00%,Amtel_Telco
2019-03-21,0.665,0.680,0.680,0.665,2.00K,0.00%,Amtel_Telco
2019-03-13,0.665,0.665,0.665,0.665,2.60K,0.00%,Amtel_Telco
2019-03-11,0.665,0.665,0.665,0.665,3.50K,0.00%,Amtel_Telco
2019-03-05,0.665,0.660,0.665,0.660,309.50K,1.53%,Amtel_Telco
2019-02-25,0.655,0.655,0.655,0.655,1.00K,0.00%,Amtel_Telco
2019-02-19,0.655,0.655,0.700,0.655,21.60K,0.00%,Amtel_Telco
2019-02-18,0.655,0.655,0.655,0.655,1.30K,0.00%,Amtel_Telco
2019-02-15,0.655,0.655,0.655,0.655,1.00K,0.00%,Amtel_Telco


In [3]:
# Rearrange columns.

stock = stock[['High','Low','Open','Price','Volume','Change%','Company']]
cols = list(stock.columns.values)
print(cols)


['High', 'Low', 'Open', 'Price', 'Volume', 'Change%', 'Company']


## Data Cleansing

In [4]:
# DATA CLEANSING - "Volume" column.

# Issue with "Volume" column. 
# It using ordinal representation (K or M) instead of numeric representation (000 or 000000), e.g. 309.50K.
# Due to that the data column type is object (should be integer).
# Implication - Can't be used for calc. or chart plotting directly.
# Need for data cleasing as below.
# Below will remove K or M and will replace it with 000 or 000000.

#print(stock.shape)     # for debugging purpose.
i = 0

while i < 931:
    # x is cell value under Volumn field.
    x = stock.iloc[i,4]
    #print(x)           # for debugging purpose.

    # K or M
    Ordinal = x[-1]
    if Ordinal == "K":
        y = x[0:-2]
        y = float(y) * 1000
        #print(y)          # for debugging purpose.
        #print(type(y))    # for debugging purpose.
    
    if Ordinal == "M":
        y = x[0:-2]
        y = float(y) * 1000000
        #print(y)          # for debugging purpose.
        #print(type(y))    # for debugging purpose.
    
    stock.ix[i, 'Volume'] = y
    i = i + 1

# The K and M has been changed to numerical representation.
# So, now we may proceed to change the "Volume" column type.

stock["Volume"] = stock["Volume"].astype(int)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [5]:
# Verification step to make sure issue on "Volume" column has been catered. 

print(stock.shape,'\n')
print(stock.info(),'\n')
stock.head()


(931, 7) 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 931 entries, 2019-03-22 to 2019-01-04
Data columns (total 7 columns):
High       931 non-null float64
Low        931 non-null float64
Open       931 non-null float64
Price      931 non-null float64
Volume     931 non-null int32
Change%    931 non-null object
Company    931 non-null object
dtypes: float64(4), int32(1), object(2)
memory usage: 74.6+ KB
None 



,High,Low,Open,Price,Volume,Change%,Company
Date,,,,,,,
2019-03-22,0.665,0.665,0.665,0.665,1000,0.00%,Amtel_Telco
2019-03-21,0.680,0.665,0.680,0.665,2000,0.00%,Amtel_Telco
2019-03-13,0.665,0.665,0.665,0.665,2600,0.00%,Amtel_Telco
2019-03-11,0.665,0.665,0.665,0.665,3500,0.00%,Amtel_Telco
2019-03-05,0.665,0.660,0.660,0.665,309500,1.53%,Amtel_Telco


In [6]:
# DATA CLEANSING - "Change%" column.

# Issue with "Change%" column. 
# At every cell, the value provided by the scrapped website is with % symbol, e.g. 1.53%.
# Due to that the data column type is object (should be float).
# Implication - Can't be used for calc. or chart plotting directly.
# Need for data cleasing as below.
# Below will remove the % symbol from every cell under "Change%" column.

#print(stock.shape)     # for debugging purpose.
i = 0

while i < 931:
    # x is cell value under "Change%" field.
    x = stock.iloc[i,5]
    #print(x)           # for debugging purpose.

    # Remove % symbol.
    y = x[0:-2]
    #print(y)          # for debugging purpose.
    #print(type(y))    # for debugging purpose.
    
    stock.ix[i, 'Change%'] = y
    i = i + 1

# The % symbol has been removed.
# So, now we may proceed to change the "Change%" column type.

stock["Change%"] = stock["Change%"].astype(float)


C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [7]:
# Verification step to make sure issue on "Change%" column has been catered. 

print(stock.shape,'\n')
print(stock.info(),'\n')
stock.head()


(931, 7) 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 931 entries, 2019-03-22 to 2019-01-04
Data columns (total 7 columns):
High       931 non-null float64
Low        931 non-null float64
Open       931 non-null float64
Price      931 non-null float64
Volume     931 non-null int32
Change%    931 non-null float64
Company    931 non-null object
dtypes: float64(5), int32(1), object(1)
memory usage: 74.6+ KB
None 



,High,Low,Open,Price,Volume,Change%,Company
Date,,,,,,,
2019-03-22,0.665,0.665,0.665,0.665,1000,0.0,Amtel_Telco
2019-03-21,0.680,0.665,0.680,0.665,2000,0.0,Amtel_Telco
2019-03-13,0.665,0.665,0.665,0.665,2600,0.0,Amtel_Telco
2019-03-11,0.665,0.665,0.665,0.665,3500,0.0,Amtel_Telco
2019-03-05,0.665,0.660,0.660,0.665,309500,1.5,Amtel_Telco


## Variance, Covariance and Correlation
We choose Digi stock history from Jan to March as example.
The stock data is obtained from web scrapping as well as from stock's historical website (Investing.com) for stock history before Feb 2019 (Remark: We started scrapping from Feb).

In [8]:
# Calculation of Variance of Every Variable.

Digi = stock.loc[stock['Company'] == 'Digi_Telco']          
# User may try with other companies stock data by changing the company name. In this case change 'Digi_Telco' to other company.
Digi[['Open', 'High', 'Low', 'Price','Volume','Change%']].var()


Open       6.790883e-03
High       6.930918e-03
Low        6.649503e-03
Price      6.084337e-03
Volume     2.803878e+12
Change%    6.231560e-01
dtype: float64

In [9]:
# Calculation of Covariance between Multivariate.

Digi[['Open', 'High', 'Low', 'Price','Volume','Change%']].cov()


,Open,High,Low,Price,Volume,Change%
Open,0.006791,0.006141,0.006134,0.005612,3.514046e+04,-0.002978
High,0.006141,0.006931,0.006159,0.005918,3.439924e+04,0.003730
Low,0.006134,0.006159,0.006650,0.006108,2.691915e+04,0.005660
Price,0.005612,0.005918,0.006108,0.006084,4.043587e+04,0.014299
Volume,35140.464407,34399.240991,26919.154579,40435.865169,2.803878e+12,512070.940123
Change%,-0.002978,0.003730,0.005660,0.014299,5.120709e+05,0.623156


### Correlation Value Strength
0.0 – 0.2 ~ Weak correlation

0.3 – 0.6 ~ Moderate correlation

0.7 – 1.0 ~ Strong correlation

In [10]:
# Calculation of Correlation between Multivariate.

Digi[['Open', 'High', 'Low', 'Price','Volume','Change%']].corr()


,Open,High,Low,Price,Volume,Change%
Open,1.000000,0.895161,0.912820,0.873024,0.254662,-0.045784
High,0.895161,1.000000,0.907167,0.911401,0.246760,0.056756
Low,0.912820,0.907167,1.000000,0.960322,0.197146,0.087920
Price,0.873024,0.911401,0.960322,1.000000,0.309585,0.232224
Volume,0.254662,0.246760,0.197146,0.309585,1.000000,0.387393
Change%,-0.045784,0.056756,0.087920,0.232224,0.387393,1.000000


In [11]:
# Plotting Correlation Matrix.

Digi = Digi[['Open', 'High', 'Low', 'Price','Volume','Change%']]
corr = Digi.corr()
corr.style.background_gradient(cmap='coolwarm')


,Open,High,Low,Price,Volume,Change%
Open,1,0.895161,0.91282,0.873024,0.254662,-0.0457844
High,0.895161,1,0.907167,0.911401,0.24676,0.0567562
Low,0.91282,0.907167,1,0.960322,0.197146,0.0879203
Price,0.873024,0.911401,0.960322,1,0.309585,0.232224
Volume,0.254662,0.24676,0.197146,0.309585,1,0.387393
Change%,-0.0457844,0.0567562,0.0879203,0.232224,0.387393,1
